In [1]:
import os
os.chdir("../")
%pwd

'/Users/shreyasshah/Desktop/kidney-disease-classification-deep-learning'

In [2]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/shreyaslshah/kidney-disease-classification-deep-learning.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'shreyaslshah'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '69ef1e71d0aed23f6e43066ca3e0337adf8b3172'

In [3]:
import tensorflow as tf

/Users/shreyasshah/Desktop/kidney-disease-classification-deep-learning/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
  path_of_model: Path
  training_data: Path
  all_params: dict
  mlflow_uri: str
  params_image_size: list
  params_batch_size: int

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
  def __init__(
          self,
          config_filepath=CONFIG_FILE_PATH,
          params_filepath=PARAMS_FILE_PATH):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    create_directories([self.config.artifacts_root])

  def get_evaluation_config(self) -> EvaluationConfig:
    eval_config = EvaluationConfig(
        path_of_model="artifacts/training/model.h5",
        training_data="artifacts/data_ingestion/kidney-ct-scan-image",
        mlflow_uri="https://dagshub.com/shreyaslshah/kidney-disease-classification-deep-learning.mlflow",
        all_params=self.params,
        params_image_size=self.params.IMAGE_SIZE,
        params_batch_size=self.params.BATCH_SIZE
    )
    return eval_config

In [8]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [13]:
class Evaluation:
  def __init__(self, config: EvaluationConfig):
    self.config = config

  def _valid_generator(self):

    datagenerator_kwargs = dict(
        rescale=1./255,
        validation_split=0.0
    )

    dataflow_kwargs = dict(
        target_size=self.config.params_image_size[:-1],
        batch_size=self.config.params_batch_size,
        interpolation="bilinear"
    )

    valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        **datagenerator_kwargs
    )

    # use the remaining training data (which is the entire dataset) for evaluation
    self.valid_generator = valid_datagenerator.flow_from_directory(
        directory=self.config.training_data,
        # subset="validation",
        shuffle=False,
        **dataflow_kwargs
    )

  @staticmethod
  def load_model(path: Path) -> tf.keras.Model:
    return tf.keras.models.load_model(path)

  def evaluation(self):
    self.model = self.load_model(self.config.path_of_model)
    self._valid_generator()
    self.score = model.evaluate(self.valid_generator)
    self.save_score()

  def save_score(self):
    scores = {"loss": self.score[0], "accuracy": self.score[1]}
    save_json(path=Path("scores.json"), data=scores)

  def log_into_mlflow(self):
    mlflow.set_registry_uri(self.config.mlflow_uri)
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    with mlflow.start_run():
      mlflow.log_params(self.config.all_params)
      mlflow.log_metrics(
          {"loss": self.score[0], "accuracy": self.score[1]}
      )
      # Model registry does not work with file store
      if tracking_url_type_store != "file":
        mlflow.keras.log_model(
            self.model, "model", registered_model_name="VGG16Model")
      else:
        mlflow.keras.log_model(self.model, "model")

In [14]:
try:
  config = ConfigurationManager()
  eval_config = config.get_evaluation_config()
  evaluation = Evaluation(eval_config)
  evaluation.evaluation()
  evaluation.log_into_mlflow()

except Exception as e:
  raise e

[2024-05-04 16:39:55,960: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-05-04 16:39:55,963: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-04 16:39:55,963: INFO: common: created directory at artifacts]
Found 465 images belonging to 2 classes.


2024-05-04 16:39:56.382721: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


30/30 [==============================] - 16s 518ms/step - loss: 0.2619 - accuracy: 0.9376
[2024-05-04 16:40:12,563: INFO: common: json file saved at: scores.json]


2024/05/04 16:40:14 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-05-04 16:40:15,017: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /var/folders/7g/xvz8wfd9477_ywln21gwjvdr0000gn/T/tmpipiikmgg/model/data/model/assets
[2024-05-04 16:40:15,522: INFO: builder_impl: Assets written to: /var/folders/7g/xvz8wfd9477_ywln21gwjvdr0000gn/T/tmpipiikmgg/model/data/model/assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/05/04 16:40:51 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 7
Created version '7' of model 'VGG16Model'.
